In [ ]:
"""
CSIRO BIOMASS - SUBMISSION NOTEBOOK (INFERENCE ONLY)
This notebook ONLY does inference - no training.
All models are loaded from pre-trained checkpoints.

Required Kaggle datasets:
1. csiro-biomass (competition data)
2. google-siglip-so400m-patch14-384 (SigLIP model)
3. My siglip_models dataset (from training notebook)
4. My dinov3_models dataset (from Colab training)

"""

import os
import gc
import random
import pickle
from pathlib import Path
from dataclasses import dataclass

import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import timm
from PIL import Image
from transformers import AutoModel, AutoImageProcessor, AutoTokenizer

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.mixture import GaussianMixture
import warnings
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# CONFIGURATION
@dataclass
class Config:
    DATA_PATH: Path = Path("/kaggle/input/csiro-biomass/")
    SIGLIP_PATH: str = "/kaggle/input/google-siglip-so400m-patch14-384/transformers/default/1"
    SIGLIP_MODELS_DIR: Path = Path("/kaggle/input/siglip-trainer")
    DINO_MODELS_DIR: Path = Path("/kaggle/input/dinov3-5fold")
    
    SEED: int = 42
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"
    
    # SigLIP settings
    PATCH_SIZE: int = 520
    OVERLAP: int = 16
    
    # DINOv3 settings
    DINO_MODEL_NAME: str = "vit_large_patch16_dinov3"
    DINO_IMG_SIZE: int = 512
    DINO_DROPOUT: float = 0.15
    DINO_BATCH_SIZE: int = 8
    
    # Targets
    TARGET_NAMES = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
    ALL_TARGET_COLS = ["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"]
    PRIMARY_TARGETS = ["Dry_Green_g", "Dry_Clover_g", "Dry_Dead_g"]
    
    TARGET_MAX = {
        "Dry_Clover_g": 71.7865,
        "Dry_Dead_g": 83.8407,
        "Dry_Green_g": 157.9836,
        "Dry_Total_g": 185.70,
        "GDM_g": 157.9836,
    }
    
    # Ensemble weights
    W_SIGLIP: float = 0.35
    W_DINO: float = 0.65
    DINO_ONLY_CLOVER: bool = True 

cfg = Config()

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(cfg.SEED)
print(f"Device: {cfg.DEVICE}")

def pivot_table_test(df: pd.DataFrame) -> pd.DataFrame:
    """Convert long format test.csv to wide format."""
    df = df.copy()
    df['target'] = 0
    df_pt = pd.pivot_table(
        df,
        values='target',
        index='image_path',
        columns='target_name',
        aggfunc='mean'              #If the data had two rows for the same image and the same target: aggfunc: mean - averages it
    ).reset_index()
    return df_pt


# PART 1: SIGLIP INFERENCE
print("\n" + "="*80)
print("PART 1: SIGLIP INFERENCE")
print("="*80)

def split_image(image: np.ndarray, patch_size: int = 520, overlap: int = 16):
    h, w, c = image.shape
    stride = patch_size - overlap
    patches = []
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            y2 = min(y + patch_size, h)
            x2 = min(x + patch_size, w)
            y1 = max(0, y2 - patch_size)
            x1 = max(0, x2 - patch_size)
            patches.append(image[y1:y2, x1:x2, :])
    return patches

def compute_siglip_embeddings(model_path: str, df: pd.DataFrame) -> np.ndarray:
    """Compute SigLIP embeddings for images."""
    print(f"Computing SigLIP embeddings for {len(df)} images...")
    
    model = AutoModel.from_pretrained(model_path, local_files_only=True).eval().to(cfg.DEVICE)
    processor = AutoImageProcessor.from_pretrained(model_path, local_files_only=True)
    
    EMBEDDINGS = []
    PATCH_BATCH = 12
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img = cv2.imread(row['image_path'])
            if img is None:
                raise ValueError(f"Image not found: {row['image_path']}")
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            patches = split_image(img, patch_size=cfg.PATCH_SIZE, overlap=cfg.OVERLAP)
            images = [Image.fromarray(p) for p in patches]
            
            feats_list = []
            with torch.no_grad():
                for i in range(0, len(images), PATCH_BATCH):
                    batch_imgs = images[i:i+PATCH_BATCH]
                    inputs = processor(images=batch_imgs, return_tensors="pt").to(cfg.DEVICE)
                    features = model.get_image_features(**inputs)
                    feats_list.append(features)
            
            features = torch.cat(feats_list, dim=0)
            avg_embed = features.mean(dim=0).detach().cpu().numpy()
            EMBEDDINGS.append(avg_embed)
            
        except Exception as e:
            print(f"Error processing: {e}")
            EMBEDDINGS.append(np.zeros(1152, dtype=np.float32))
    
    del model, processor
    torch.cuda.empty_cache()
    gc.collect()
    
    return np.stack(EMBEDDINGS)

def generate_semantic_features(image_embeddings_np: np.ndarray, model_path: str) -> np.ndarray:
    """Generate semantic features using text probing."""
    print("Generating semantic features...")
    
    model = AutoModel.from_pretrained(model_path, local_files_only=True).to(cfg.DEVICE)
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
    
    concept_groups = {
        "bare": ["bare soil", "dirt ground", "sparse vegetation", "exposed earth"],
        "sparse": ["low density pasture", "thin grass", "short clipped grass"],
        "medium": ["average pasture cover", "medium height grass", "grazed pasture"],
        "dense": ["dense tall pasture", "thick grassy volume", "high biomass", "overgrown vegetation"],
        "green": ["lush green vibrant pasture", "photosynthesizing leaves", "fresh growth"],
        "dead": ["dry brown dead grass", "yellow straw", "senesced material", "standing hay"],
        "clover": ["white clover", "trifolium repens", "broadleaf legume", "clover flowers"],
        "grass": ["ryegrass", "blade-like leaves", "fescue", "grassy sward"],
    }
    
    concept_vectors = {}
    with torch.no_grad():
        for name, prompts in concept_groups.items():
            inputs = tokenizer(prompts, padding="max_length", return_tensors="pt").to(cfg.DEVICE)
            emb = model.get_text_features(**inputs)
            emb = emb / emb.norm(p=2, dim=-1, keepdim=True)
            concept_vectors[name] = emb.mean(dim=0, keepdim=True)
    
    img_tensor = torch.tensor(image_embeddings_np, dtype=torch.float32).to(cfg.DEVICE)
    img_tensor = img_tensor / img_tensor.norm(p=2, dim=-1, keepdim=True)
    
    scores = {}
    for name, vec in concept_vectors.items():
        scores[name] = torch.matmul(img_tensor, vec.T).detach().cpu().numpy().flatten()
    
    df_scores = pd.DataFrame(scores)
    df_scores['ratio_greenness'] = df_scores['green'] / (df_scores['green'] + df_scores['dead'] + 1e-6)
    df_scores['ratio_clover'] = df_scores['clover'] / (df_scores['clover'] + df_scores['grass'] + 1e-6)
    df_scores['ratio_cover'] = (df_scores['dense'] + df_scores['medium']) / (df_scores['bare'] + df_scores['sparse'] + 1e-6)
    
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    
    return df_scores.values.astype(np.float32)

def run_siglip_inference(test_df_wide):
    """Run SigLIP inference using pre-trained models."""
    
    print("Loading pre-trained SigLIP models...")
    
    with open(cfg.SIGLIP_MODELS_DIR / "feature_engines.pkl", "rb") as f:
        all_engines = pickle.load(f)
    print(f"  Loaded {len(all_engines)} feature engines")
    
    with open(cfg.SIGLIP_MODELS_DIR / "models.pkl", "rb") as f:
        all_models = pickle.load(f)
    print(f"  Loaded models for {len(all_models)} folds")
    
    with open(cfg.SIGLIP_MODELS_DIR / "config.pkl", "rb") as f:
        saved_config = pickle.load(f)
    
    test_embeddings = compute_siglip_embeddings(cfg.SIGLIP_PATH, test_df_wide)
    
    sem_test = generate_semantic_features(test_embeddings, cfg.SIGLIP_PATH)
    
    print("Running SigLIP inference...")
    
    target_max_arr = np.array([cfg.TARGET_MAX[t] for t in cfg.TARGET_NAMES], dtype=np.float32)
    n_folds = saved_config['N_FOLDS']
    model_names = saved_config['model_names']
    train_targets = saved_config['TRAIN_TARGETS']
    
    y_pred_test_accum = np.zeros((len(test_df_wide), len(cfg.TARGET_NAMES)), dtype=np.float32)
    
    for fold in range(n_folds):
        print(f"  Fold {fold}...")
        
        engine = all_engines[fold]
        # Semantic features are normalized using their OWN mean/std inside transform()
        # This matches the training code behavior
        x_te_eng = engine.transform(test_embeddings, X_semantic=sem_test)
        
        fold_pred = np.zeros((len(test_df_wide), len(cfg.TARGET_NAMES)), dtype=np.float32)
        
        for model_name in model_names:
            model_pred = np.zeros((len(test_df_wide), len(cfg.TARGET_NAMES)), dtype=np.float32)
            
            for target_name in cfg.TARGET_NAMES:
                k = cfg.TARGET_NAMES.index(target_name)
                
                if target_name == 'Dry_Clover_g':
                    model_pred[:, k] = 0.0
                else:
                    model = all_models[fold][model_name][target_name]
                    pred = model.predict(x_te_eng).astype(np.float32)
                    model_pred[:, k] = pred * target_max_arr[k]
            
            fold_pred += model_pred
        
        fold_pred /= len(model_names)  # Average across model types
        y_pred_test_accum += fold_pred
    
    y_pred_test_accum /= n_folds  # Average across folds
    
    # Create result DataFrame with image_id
    result_df = pd.DataFrame()
    result_df['image_id'] = test_df_wide['image_path'].apply(lambda p: Path(p).stem)
    
    for i, col in enumerate(cfg.TARGET_NAMES):
        result_df[col] = y_pred_test_accum[:, i]
    
    # Post-process
    result_df['Dry_Clover_g'] = 0.0
    result_df['GDM_g'] = result_df['Dry_Green_g'] + result_df['Dry_Clover_g']
    result_df['Dry_Total_g'] = result_df['GDM_g'] + result_df['Dry_Dead_g']
    
    for col in cfg.TARGET_NAMES:
        result_df[col] = result_df[col].clip(lower=0.0)
    
    print(f"SigLIP predictions: {len(result_df)} images")
    
    # Cleanup
    del all_engines, all_models, test_embeddings, sem_test
    gc.collect()
    torch.cuda.empty_cache()
    
    return result_df

# PART 2: DINOV3 INFERENCE
print("\n" + "="*80)
print("PART 2: DINOV3 INFERENCE")
print("="*80)

def clean_image(img):
    """Remove bottom 10% and inpaint orange date stamps."""
    h, w = img.shape[:2]
    img = img[0:int(h * 0.90), :]
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, np.array([5, 150, 150]), np.array([25, 255, 255]))
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2)
    if np.sum(mask) > 0:
        img = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)
    
    return img

class FiLM(nn.Module):
    def __init__(self, feat_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feat_dim, feat_dim // 2),
            nn.GELU(),
            nn.Linear(feat_dim // 2, feat_dim * 2),
        )

    def forward(self, context):
        gamma_beta = self.mlp(context)
        gamma, beta = torch.chunk(gamma_beta, 2, dim=1)
        return gamma, beta

class CSIROModelRegressor(nn.Module):
    def __init__(self, model_name, dropout=0.2, pretrained=False):
        super().__init__()
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,  # Always False for inference
            num_classes=0,
            global_pool="avg",
        )
        nf = self.backbone.num_features
        
        self.film = FiLM(nf)
        
        self.head = nn.Sequential(
            nn.Linear(nf * 3, 512),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(128, 3),
        )
        self.softplus = nn.Softplus(beta=1.0)
        
        self.register_buffer('max_green', torch.tensor(cfg.TARGET_MAX["Dry_Green_g"]))
        self.register_buffer('max_clover', torch.tensor(cfg.TARGET_MAX["Dry_Clover_g"]))
        self.register_buffer('max_dead', torch.tensor(cfg.TARGET_MAX["Dry_Dead_g"]))

    def forward(self, full_img, left_img, right_img):
        full_feat = self.backbone(full_img)
        left_feat = self.backbone(left_img)
        right_feat = self.backbone(right_img)
        
        gamma, beta = self.film(full_feat)
        left_mod = left_feat * (1.0 + gamma) + beta
        right_mod = right_feat * (1.0 + gamma) + beta
        
        comb = torch.cat([full_feat, left_mod, right_mod], dim=1)
        
        prim_norm = self.softplus(self.head(comb))
        green_norm = prim_norm[:, 0:1]
        clover_norm = prim_norm[:, 1:2]
        dead_norm = prim_norm[:, 2:3]
        
        green = green_norm * self.max_green
        clover = clover_norm * self.max_clover
        dead = dead_norm * self.max_dead
        
        gdm = green + clover
        total = gdm + dead
        
        # Pack: [Green, Dead, Clover, GDM, Total]
        packed = torch.cat([green, dead, clover, gdm, total], dim=1)
        return packed

class BiomassTestDataset(Dataset):
    def __init__(self, df, data_root, transform):
        self.df = df.reset_index(drop=True)
        self.data_root = str(data_root)
        self.transform = transform
        self.paths = self.df["image_path"].values

    def __len__(self):
        return len(self.df)

    def _read_image(self, rel_path):
        abs_path = rel_path if os.path.isabs(rel_path) else os.path.join(self.data_root, rel_path)
        img = cv2.imread(abs_path)
        if img is None:
            print(f"Warning: Could not read image: {abs_path}")
            img = np.zeros((1000, 2000, 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    def __getitem__(self, idx):
        rel_path = self.paths[idx]
        img = self._read_image(rel_path)
        img = clean_image(img)
        
        h, w, _ = img.shape
        mid = w // 2
        left = img[:, :mid]
        right = img[:, mid:]
        full = img
        
        full = self.transform(image=full)["image"]
        left = self.transform(image=left)["image"]
        right = self.transform(image=right)["image"]
        return full, left, right

def get_val_transforms():
    return A.Compose([
        A.Resize(height=cfg.DINO_IMG_SIZE, width=cfg.DINO_IMG_SIZE),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

@torch.no_grad()
def predict_one_dino_model(ckpt_path, loader):
    """Run inference with one DINO checkpoint."""
    model = CSIROModelRegressor(
        cfg.DINO_MODEL_NAME, 
        dropout=cfg.DINO_DROPOUT, 
        pretrained=False  # Always False - we load our own weights
    ).to(cfg.DEVICE)
    
    state = torch.load(ckpt_path, map_location=cfg.DEVICE)
    model.load_state_dict(state, strict=True)
    model.eval()
    
    preds = []
    
    # FIX: Handle CPU gracefully
    use_amp = (cfg.DEVICE == "cuda")
    
    for full, left, right in tqdm(loader, desc=f"Infer {os.path.basename(ckpt_path)}", leave=False):
        full = full.to(cfg.DEVICE, non_blocking=True)
        left = left.to(cfg.DEVICE, non_blocking=True)
        right = right.to(cfg.DEVICE, non_blocking=True)
        
        if use_amp:
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=True):
                out = model(full, left, right)
        else:
            out = model(full, left, right)
        
        preds.append(out.float().cpu().numpy())
    
    preds = np.concatenate(preds, axis=0)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
    return preds

def run_dino_inference(test_df_wide):
    """Run DINOv3 inference using pre-trained models."""
    
    test_ds = BiomassTestDataset(test_df_wide, cfg.DATA_PATH, get_val_transforms())
    test_loader = DataLoader(
        test_ds,
        batch_size=cfg.DINO_BATCH_SIZE * 2,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        drop_last=False,
    )
    
    # Find checkpoints
    print("Looking for DINO checkpoints...")
    ckpts = []
    for fold in range(5):
        ckpt = cfg.DINO_MODELS_DIR / f"best_model_fold{fold}.pth"
        if ckpt.exists():
            ckpts.append(str(ckpt))
            print(f"  Found: {ckpt}")
    
    if len(ckpts) == 0:
        raise FileNotFoundError(f"No DINO checkpoints found in {cfg.DINO_MODELS_DIR}")
    
    # Predict each fold and average
    print(f"Running DINO inference with {len(ckpts)} checkpoints...")
    fold_preds = []
    for ckpt in ckpts:
        fold_preds.append(predict_one_dino_model(ckpt, test_loader))
    
    pred_mean = np.mean(np.stack(fold_preds, axis=0), axis=0)
    
    result_df = pd.DataFrame()
    result_df['image_id'] = test_df_wide['image_path'].apply(lambda p: Path(p).stem)
    result_df['Dry_Green_g'] = pred_mean[:, 0]
    result_df['Dry_Dead_g'] = pred_mean[:, 1]
    result_df['Dry_Clover_g'] = pred_mean[:, 2]
    result_df['GDM_g'] = pred_mean[:, 3]
    result_df['Dry_Total_g'] = pred_mean[:, 4]
    
    print(f"DINO predictions: {len(result_df)} images")
    
    return result_df

# PART 3: ENSEMBLE + SUBMISSION
def ensemble_and_submit(siglip_preds, dino_preds, test_df_raw):

    print("\n" + "="*80)
    print("PART 3: ENSEMBLE + SUBMISSION")
    print("="*80)
    
    print("Merging predictions on image_id...")
    
    merged = siglip_preds[['image_id']].merge(
        dino_preds[['image_id'] + cfg.PRIMARY_TARGETS],
        on='image_id',
        how='inner',
        suffixes=('', '_dino')
    )
    
    merged = merged.merge(
        siglip_preds[['image_id'] + cfg.PRIMARY_TARGETS],
        on='image_id',
        how='inner',
        suffixes=('_dino', '_sig')
    )
    
    # Sanity check
    if len(merged) != len(siglip_preds) or len(merged) != len(dino_preds):
        print(f"WARNING: Row count mismatch! SigLIP={len(siglip_preds)}, DINO={len(dino_preds)}, Merged={len(merged)}")
    
    print(f"Merged {len(merged)} images")
    
    # Ensemble primary targets
    result = merged[['image_id']].copy()
    
    for col in cfg.PRIMARY_TARGETS:
        dino_col = f"{col}_dino"
        sig_col = f"{col}_sig"
        
        if cfg.DINO_ONLY_CLOVER and col == "Dry_Clover_g":
            result[col] = merged[dino_col]
            print(f"  {col}: DINO only")
        else:
            result[col] = cfg.W_DINO * merged[dino_col] + cfg.W_SIGLIP * merged[sig_col]
            print(f"  {col}: {cfg.W_DINO:.0%} DINO + {cfg.W_SIGLIP:.0%} SigLIP")
    
    # Derive GDM and Total
    result['GDM_g'] = result['Dry_Green_g'] + result['Dry_Clover_g']
    result['Dry_Total_g'] = result['GDM_g'] + result['Dry_Dead_g']
    
    # Clip negatives
    for col in cfg.TARGET_NAMES:
        result[col] = result[col].clip(lower=0.0)
    
    print("\nBuilding submission from test.csv template...")
    
    test_template = test_df_raw.copy()
    test_template['image_id'] = test_template['sample_id'].str.rsplit('__', n=1).str[0]
    test_template['target_name'] = test_template['sample_id'].str.rsplit('__', n=1).str[1]
    
    # Merge predictions onto template
    final = test_template.merge(result, on='image_id', how='left')
    
    # Pick the correct target value for each row
    def get_target_value(row):
        target_name = row['target_name']
        return row.get(target_name, np.nan)
    
    final['target'] = final.apply(get_target_value, axis=1)
    
    # Final submission
    submission = final[['sample_id', 'target']].copy()
    
    #SANITY CHECKS
    print("\n--- Sanity Checks ---")
    print(f"Test rows (expected): {len(test_df_raw)}")
    print(f"Submission rows: {len(submission)}")
    print(f"Missing values: {submission['target'].isna().sum()}")
    print(f"Columns: {submission.columns.tolist()}")
    
    assert len(submission) == len(test_df_raw), "Row count mismatch!"
    assert submission['target'].isna().sum() == 0, "Missing predictions!"
    assert submission.columns.tolist() == ['sample_id', 'target'], "Wrong columns!"
    
    # Save
    submission.to_csv('submission.csv', index=False)
    print(f"\n✓ Saved submission.csv with {len(submission)} rows")
    print("\nFirst 10 rows:")
    print(submission.head(10))
    
    print("\nStats:")
    print(submission['target'].describe())
    
    return submission

class SupervisedEmbeddingEngine:
    """
    MATCHES TRAINING EXACTLY.
    """
    def __init__(self, n_pca=0.80, n_pls=8, n_gmm=6, random_state=42):
        self.scaler = StandardScaler()
        self.pca = PCA(n_components=n_pca, random_state=random_state)
        self.pls = PLSRegression(n_components=n_pls, scale=False)
        self.gmm = GaussianMixture(n_components=n_gmm, covariance_type='diag', random_state=random_state)
        self.pls_fitted_ = False

    def fit(self, X, y=None, X_semantic=None):
        # Scale
        X_scaled = self.scaler.fit_transform(X)
        
        # Unsupervised
        self.pca.fit(X_scaled)
        self.gmm.fit(X_scaled)
        
        # Supervised
        if y is not None:
            self.pls.fit(X_scaled, y)
            self.pls_fitted_ = True
        return self

    def transform(self, X, X_semantic=None):
        X_scaled = self.scaler.transform(X)
        
        features = [self.pca.transform(X_scaled)]
        
        if self.pls_fitted_:
            features.append(self.pls.transform(X_scaled))
            
        features.append(self.gmm.predict_proba(X_scaled))
        
        if X_semantic is not None:
            sem_norm = (X_semantic - np.mean(X_semantic, axis=0)) / (np.std(X_semantic, axis=0) + 1e-6)
            features.append(sem_norm)
            
        return np.hstack(features)

# MAIN EXECUTION
if __name__ == "__main__":
    
    print("\n" + "="*80)
    print("LOADING TEST DATA")
    print("="*80)
    
    test_df_raw = pd.read_csv(cfg.DATA_PATH / "test.csv")
    print(f"Test rows (raw): {len(test_df_raw)}")
    
    # Convert to wide format for inference
    test_df_wide = pivot_table_test(test_df_raw)
    
    test_df_wide['image_path'] = test_df_wide['image_path'].apply(
        lambda x: str(cfg.DATA_PATH / "test" / os.path.basename(x))
    )
    print(f"Test images: {len(test_df_wide)}")
    
    for idx, row in test_df_wide.iterrows():
        if not os.path.exists(row['image_path']):
            print(f"WARNING: Image not found: {row['image_path']}")
    
    # Run SigLIP
    siglip_preds = run_siglip_inference(test_df_wide)
    
    # Run DINO
    dino_preds = run_dino_inference(test_df_wide)
    
    # Ensemble and submit
    submission = ensemble_and_submit(siglip_preds, dino_preds, test_df_raw)
    
    print("\n" + "="*80)
    print("DONE!")
    print("="*80)

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Device: cuda

PART 1: SIGLIP INFERENCE

PART 2: DINOV3 INFERENCE

LOADING TEST DATA
Test rows (raw): 5
Test images: 1
Loading pre-trained SigLIP models...
  Loaded 5 feature engines
  Loaded models for 5 folds
Computing SigLIP embeddings for 1 images...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


Generating semantic features...
Running SigLIP inference...
  Fold 0...
  Fold 1...
  Fold 2...
  Fold 3...
  Fold 4...
SigLIP predictions: 1 images
Looking for DINO checkpoints...
  Found: /kaggle/input/dinov3-5fold/best_model_fold0.pth
  Found: /kaggle/input/dinov3-5fold/best_model_fold1.pth
  Found: /kaggle/input/dinov3-5fold/best_model_fold2.pth
  Found: /kaggle/input/dinov3-5fold/best_model_fold3.pth
  Found: /kaggle/input/dinov3-5fold/best_model_fold4.pth
Running DINO inference with 5 checkpoints...


DINO predictions: 1 images

PART 3: ENSEMBLE + SUBMISSION
Merging predictions on image_id...
Merged 1 images
  Dry_Green_g: 65% DINO + 35% SigLIP
  Dry_Clover_g: DINO only
  Dry_Dead_g: 65% DINO + 35% SigLIP

Building submission from test.csv template...

--- Sanity Checks ---
Test rows (expected): 5
Submission rows: 5
Missing values: 0
Columns: ['sample_id', 'target']

✓ Saved submission.csv with 5 rows

First 10 rows:
                    sample_id     target
0  ID1001187975__Dry_Clover_g   2.464985
1    ID1001187975__Dry_Dead_g  30.822813
2   ID1001187975__Dry_Green_g  31.321903
3   ID1001187975__Dry_Total_g  64.609703
4         ID1001187975__GDM_g  33.786888

Stats:
count     5.000000
mean     32.601258
std      22.016703
min       2.464985
25%      30.822813
50%      31.321903
75%      33.786888
max      64.609703
Name: target, dtype: float64

DONE!
